# Εισαγωγή στη Βαθιά Μάθηση με PyTorch

Σε αυτό το σημειωματάριο, γίνεται η εισαγωγή στη βιβλιοθήκη [PyTorch](http://pytorch.org/), ένα σύνολο απο εργαλεία για την δημιουργία και εκπαίδευση Νευρωνικών Δικτύων. Η PyTorch κατά γενικό κανόνα παρουσιάζει πολλές ομοιότητες με τη γνωστή πλέον σε εσάς Numpy. Στην ουσία, όλοι οι πίνακες της Numpy, μπορούν να θεωρηθούν τανυστές. Η PyTorch δέχεται αυτούς τους τενσορες και με έναν εύκολο τρόπο τους διοχετεύει στη GPU για την πιο γρήγορη επεξεργασία τους, η οποία είναι απαραίτητη κατά την εκπαίδευση ενός δικτύου. Παρέχει επίσης συναρτήσεις για τον αυτόματο υπολογισμό των μερικών παραγώγων (για το βήμα του backpropagation!) και άλλες συναρτήσεις ειδικά για την δημιουργία της αρχιτεκτονικής των νευρωνικών δικτύων. Όλα μαζί αποτελούν τη PyTorch η οποία καταλήγει να είναι μία απόλυτα συμβατή βιβλιοθήκη με τη Pyhton και τη Numpy/Scipy σε σύγκριση με άλλες βιβλιοθήκες όπως η TensorFlow και άλλα frameworks.



## Νευρωνικά Δίκτυα

Η βαθιά μάθηση βασίζεται στα τεχνητά νευρωνικά δίκτυα που πρωτοχρησιμοποιήθηκαν στην αρχική τους μορφή από τα τέλη της δεκαετίας του 1950. Τα δίκτυα αποτελούνται από μεμονωμένα δομικά στοιχεία που προσεγγίζουν τους νευρώνες, και συνήθως ονομάζονται μονάδες (units) ή "νευρώνες". Κάθε νευρώνας έχει έναν αριθμό σταθμισμένων εισόδων. Αυτές οι σταθμισμένες είσοδοι αθροίζονται μαζί (με βάση ένα γραμμικό συνδυασμό) και στη συνέχεια περνούν από μία συνάρτηση ενεργοποίησης για να δώσουν την έξοδο του νευρώνα.

<img src="assets/simple_neuron.png" width=400px>

Η μαθηματική σχέση αυτή αντιστοιχεί στο παρακάτω: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

Σε μορφή διανυσμάτων, αυτό αντιστοιχεί στο εσωτερικό γινόμενο (dot/inner product) των δύο διανυσμάτων:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Τανυστές

Αποδεικνύεται ότι οι υπολογισμοί στα νευρωνικά δίκτυα είναι απλώς μια σειρά πράξεων γραμμικής άλγεβρας πάνω στους *τανυστές*, που αποτελούν μια γενίκευση των πινάκων. Ένα διάνυσμα είναι ένας μονοδιάστατος τανυστής, ένας πίνακας είναι ένας δισδιάστατος τανυστής, ένας πίνακας με τρεις δείκτες είναι ένας τρισδιάστατος τανυστής (για παράδειγμα, έγχρωμες εικόνες RGB). Η βασική δομή δεδομένων για τα νευρωνικά δίκτυα είναι οι τανυστές και η PyTorch (καθώς και σχεδόν κάθε άλλη βιβλιοθήκη βαθιάς μάθησης) είναι χτισμένη γύρω από τους τανυστές.

<img src="assets/tensor_examples.svg" width=600px>

Έχοντας καλύψει τα βασικά, ήρθε η ώρα να διερευνήσουμε πώς μπορούμε να χρησιμοποιήσουμε τη PyTorch για να οικοδομήσουμε ένα απλό νευρωνικό δίκτυο.

In [1]:
# First, import PyTorch
import torch

In [2]:
def activation(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [3]:
### Δημιουργώ τυχαία δεδομένα
torch.manual_seed(7) # Ορίζω random seed για να μπορω να κανω ευκολα debug

# Τα δεδομένα εισόδου είναι 5 τυχαίες τιμές μίας κανονικής κατανομής
features = torch.randn((1, 5))
# Τα βάρη για τα δεδομένα εισόδου, πάλι απο κανονική κατανομή. Προσοχή στο _like(features)
weights = torch.randn_like(features)
# και τιμή για το bias
bias = torch.randn((1, 1))
print(features)
print(weights)

tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]])
tensor([[-0.8948, -0.3556,  1.2324,  0.1382, -1.6822]])


Στις πάνω γραμμές κώδικα δημιουργήσαμε τυχαίες τιμές για να μπορέσουμε να πάρουμε τα αποτελέσματα απο ένα απλό δίκτυο. Οι τιμές είναι τυχαίες προς το παρών, αλλά όσο προχωρούμε θα αρχίζουμε να χρησιμοποιούμε κανονικά δεδομένα. Αναλύωντας τη κάθε γραμμή κώδικα:

`features = torch.randn((1, 5))` δημιουργεί ένα τανυστή με μέγεθος (shape) `(1, 5)`, μία γραμμή και πέντε στήλες, ο οποίος περιέχει τυχαίες τιμές που ακολουθούν μία κανονική κατανομή μεσης τιμής μηδέν και τυπικής απόκλισης ένα. 

`weights = torch.randn_like(features)` δημιουργεί έναν άλλο τανυστή με μέγεθος ίδιο με τον τανυστή `features`, με τιμές ξάνα κανονικής κατανομής.

Τέλος, η γραμμή κώδικα `bias = torch.randn((1, 1))` δημιουργεί μία τιμή απο μία κανονική κατανομή.

Οι τανυστές στη PyTorch μπορούν να προστεθούν, να πολλαπλασιαστούν, να αφαιρεθούν κ.λπ., όπως οι αριθμητικοί πίνακες στη Numpy. Γενικά, θα χρησιμοποιείτε τους τανυστές στη PyTorch με τον ίδιο τρόπο που θα χρησιμοποιούσατε και τους αριθμητικούς πίνακες. Παρουσιάζουν όμως κάποια ωραία πλεονεκτήματα, όπως η επιτάχυνση σε GPU που θα δούμε αργότερα. Προς το παρόν, χρησιμοποιήστε τα δεδομένα που δημιουργήσατε για να υπολογίσετε την έξοδο του απλού νευρωνικού δικτύου.

 
> **Άσκηση**: Υπολογίστε την έξοδο του δικτύου με είσοδο τα `features`, βάρη τα `weights`, και πόλωση/κατώφλι `bias`. Όπως στη Numpy, και η PyTorch έχει την συνάρτηση [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum), όπως και τη μέθοδο `.sum()` για τους τανυστές, για να υπολογίζονται αθροίσματα. Χρησιμοποιήστε τη συνάρτηση `activation` όπως ορίστηκε παραπάνω ως συνάρτηση ενεργοποίησης.

In [5]:
### Λ΄ύση

# Υπολογίζω την έξοδο (labels) απο τα δεδομένα εισόδου και τα βάρη

y = activation(torch.sum(features * weights) + bias)
y = activation((features * weights).sum() + bias)


Μπορείτε να κάνετε τον πολλαπλασιασμό και το άθροισμα με μία πράξη χρησιμοποιώντας πολλαπλασιασμό πινάκων. Γενικά, είναι καλό και πρέπει να χρησιμοποιείται πολλαπλασιασμούς πινάκων επειδή είναι πιο αποδοτικοί και μπορούν να επιταχυνθούν χρησιμοποιώντας σύγχρονες βιβλιοθήκες και υπολογιστές υψηλών επιδόσεων σε GPU επεξεργαστές.

Εδώ, θέλουμε να κάνουμε ένα πολλαπλασιαμό πινάκων μεταξύ των features και των weights. Για την πράξη αυτή μπορούμε να χρησιμοποιήσουμε την εντολή [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) ή την [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul) που είναι λίγο πιο πολύπλοκη και υποστηρίζει broadcasting. Αν προσπαθήσουμε να εκτελέσουμε αυτή τη πράξη με τα `features` και τα `weights` όπως είναι αυτή τη στιγμή, θα δούμε το παρακάτω σφάλμα

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```
Καθώς θα δημιουργείτε νευρωνικά δίκτυα σε οποιοδήποτε library, το σφάλμα αυτό θα το βλέπετε συχνά. Πολύ συχνά. Αυτό που συμβαίνει γιατί οι τανυστές μας δεν έχουν τα σωστά shapes για να εκτελεστεί ο πολλαπλασιαμός πινάκων. Να θυμάστε ότι για τον πολλαπλασιασμό πινάκων, ο αριθμός των στηλών στον πρώτο τανυστή πρέπει να είναι ίσος με τον αριθμό των γραμμών του δεύτερου τανυστή. Και τα `features` και τα `weights` έχουν το ίδιο σχήμα, `(1, 5)`. Αυτό σημαίνει ότι πρέπει να αλλάξουμε τη μορφή του πίνακα `weights` για να εκτελέσουμε σωστά την πράξη του πολλαπλασιασμού πινάκων.

**Note:** Για να δείτε τη μορφή (shape) ενός τανυστή με όνομα `tensor`, εκτελέστε `tensor.shape`. Εάν δημιουργείτε νευρωνικά δίκτυα, θα χρησιμοποιείτε συχνά αυτή τη μέθοδο.

Υπάρχουν μερικές επιλογές εδώ: [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), and [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

* `weights.reshape(a, b)` θα επιστρέψει έναν νέο τανυστή με τα ίδια δεδομένα με το `weights` με μέγεθος `(a, b)` καποιες φορές, και μερικές φορές ως κλώνο, καθώς σε αυτό θα αντιγράφει τα δεδομένα σε ένα άλλο μέρος της μνήμης.
* `weights.resize_(a, b)` επιστρέφει τον ίδιο τανυστή με διαφορετικό σχήμα. Ωστόσο, αν το νέο σχήμα έχει λιγότερα στοιχεία από τον αρχικό τανυστή, κάποια στοιχεία θα αφαιρεθούν από τον τανυστή (αλλά όχι από τη μνήμη). Αν το νέο σχήμα έχει περισσότερα στοιχεία από τον αρχικό τανυστή, τα νέα στοιχεία δεν θα αρχικοποιηθούν στη μνήμη. Εδώ πρέπει να σημειώσω ότι η υπογράμμιση στο τέλος της μεθόδου υποδηλώνει ότι αυτή η μέθοδος εκτελείται **in-place**. Εδώ είναι ένα μεγάλο thread σε ένα φόρουμ για να δείτε λεπτομέριες [read more about in-place operations](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) in PyTorch.
* `weights.view(a, b)` θα επιστρέψει έναν νέο τανυστή με τα ίδια δεδομένα με το `weights` με μέγεθος `(a, b)`.

Συνήθως χρησιμοποιώ τη μέθοδο `.view()`, αλλά οποιαδήποτε από τις τρεις μεθόδους θα λειτουργήσει σε αυτή τη περίπτωση. Έτσι, τώρα μπορώ να ΄΄αξω τη μορφή του πίνακα `weights` έτσι ώστε να έχει πέντε γραμμές και μία στήλα με μία εντολή όπως αυτή `weights.view(5, 1)`.

> **Άσκηση**: Υπολογίστε την έξοδο του μικρού μας δικτύου χρησιμοποιώντας πολλαπλασιασμό πινάκων.

In [ ]:
## Λ΄ύση

y = activation(torch.mm(features, weights.view(5,1)) + bias)

### Ας τα συνδιάσουμε όλα!

Με την παραπάνω εντολή μπορείτε να υπολογίσετε την έξοδο για έναν μόνο νευρώνα. Η πραγματική ισχύς αυτού του αλγορίθμου εμφανίζεται όταν αρχίζουμε να στοιβάζουμε όλους αυτούς τους επιμέρους κόμβους σε επίπεδα και στοίβες επιπέδων σε ένα δίκτυο νευρώνων. Η έξοδος ενός επιπέδου γίνεται η είσοδος για το επόμενο επίπεδο. Με αυτές τις πολλαπλές μονάδες εισόδου και μονάδες εξόδου, πρέπει τώρα να εκφράσουμε τα βάρη ως πίνακα.

<img src='assets/multilayer_diagram_weights.png' width=450px>

Το πρώτο επίπεδο που εμφανίζεται στο κάτω μέρος είναι οι είσοδοι, που κατά κανόνα ονομάζεται **επίπεδο εισόδου**. Το μεσαίο επίπεδο ονομάζεται **κρυφό επίπεδο** και το τελικό επίπεδο (πάνω) είναι το **επίπεδο εξόδου**. Μπορούμε εκ νέου να εκφράσουμε αυτό το δίκτυο μαθηματικά με πίνακες και να χρησιμοποιήσουμε πολλαπλασιασμούς πινάκων για να πάρουμε τους γραμμικούς συνδυασμούς τους για κάθε κόμβο σε μία εντολή. Για παράδειγμα, μπορεί να υπολογιστεί το κρυφό επίπεδο ($h_1$ και $h_2$) ως:


$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

Η έξοδος για αυτό το μικρό δίκτυο υπολογίζεται αν λαβω το κρυφό επίπεδο σαν είσοδο του επιπέδου εξόδου. Η έξοδος του δικτύου εκφράζεται απλά:

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [ ]:
### Δημιούργησε δεδομ΄ένα
torch.manual_seed(7) # Ορίζω random seed για να μπορω να κανω ευκολα debug

# Η είσοδος είναι 3 τυχαίες μεταβλητές απο μία κανονική κατανομή
features = torch.randn((1, 3))

# Καθορίστε το μέγεθος κάθε επιπέδου στο δίκτυό μας
n_input = features.shape[1]     # Ο αριθμός των κόμβων εισόδου πρέπει να ταιριάζει με τον αριθμό των χαρακτηριστικών εισόδου
n_hidden = 2                    # Αριθμός κόμβων κρυφού επιπέδου 
n_output = 1                    # Αριθμός κ΄όμβων εξόδου

# Τα βάρη μεταξύ του επιπέδου εισόδου και του κρυφού επιπέδου
W1 = torch.randn(n_input, n_hidden)
# Τα βάρη μεταξύ του κρυφού επι΄΄πέδου και του επι΄΄πεδου εξόδου
W2 = torch.randn(n_hidden, n_output)

# και οι τιμές των bias για τα κρυφό επίδεδο και το επίπεδο εξόδου 
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

> **Άσκηση:** Υπολογίστε την έξοδο για αυτό το δίκτυο πολλών επιπέδων χρησιμοποιώντας τα βάρη `W1` & `W2`, και τις πολώσεις, `B1` & `B2`. 

In [ ]:
### Λ΄υση

h = activation(torch.mm(features, W1) + B1)
print(h)
output = activation(torch.mm(h, W2) + B2)
print(output)

Εάν το κάνατε σωστά, θα πρέπει να δείτε την έξοδο `tensor([[ 0.3171]])`.

Ο αριθμός των κρυφών επιπέδων είναι μια παράμετρος του δικτύου, που ονομάζεται συχνά **υπερπαράμετρος** για να  διαφοροποιήθει από τις παραμέτρους των βαρών και πόλωσης. Όπως θα δούμε αργότερα, όταν συζητάμε για την εκπαίδευση ενός νευρικού δικτύου, όσο πιο πολλούς κόμβους έχει ένα δίκτυο και όσο περισσότερα επίπεδα, τόσο πιο ικανά είναι να μάθει από τα δεδομένα και να κάνει ακριβείς προβλέψεις.

## Numpy σε Torch και πίσω

Η PyTorch έχει ένα ωραίο χαρακτηριστικό για τη μετατροπή μεταξύ πινάκων Numpy και Torch τανυστών. Για να δημιουργήσετε έναν τανυστή από έναν πίνακα Numpy, εκτελέστε `torch.from_numpy()`. Για να μετατρέψετε έναν τανυστή σε έναν πίνακα Numpy, χρησιμοποιείστε τη μέθοδο `.numpy()`.

In [ ]:
import numpy as np
a = np.random.rand(4,3)
a

In [ ]:
b = torch.from_numpy(a)
b

In [ ]:
b.numpy()

Η μνήμη είναι κοινή μεταξύ του αριθμητικού πίνακα και του τανυστή Torch, οπότε αν αλλάξετε τις τιμές του ενός αντικειμένου, θα αλλαξουν και οι τιμές του άλλου.

In [ ]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

In [ ]:
# Numpy array matches new values from Tensor
a

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

Αυτό το notebook 📖 δημιουργήθηκε για το μάθημα ***Υπολογιστική Νοημοσύνη και Μηχανική Μάθηση*** του Τμήματος Μηχανικών Παραγωγής και Διοίκησης, της Πολυτεχνικής Σχολής του Δημοκριτείου Πανεπιστημίου Θράκης.<br>
This notebook is made available under the Creative Commons Attribution [(CC-BY)](https://creativecommons.org/licenses/by/4.0/legalcode) license. Code is also made available under the [MIT License](https://opensource.org/licenses/MIT).<br>
Author: Asst. Prof. Angelos Amanatiadis
<img src="assets/cc.png" style="width:55px; float: right; margin: 0px 0px 0px 0px;"></img>
<img src="assets/mit.png" style="width:40px; float: right; margin: 0px 10px 0px 0px;"></img>